# Insider Trading Data Pipeline



Simple ETL pipeline to fetch insider transactions from Finnhub API and load into MySQL.

## Prerequisites
Set these secrets in Google Colab:
- `FINNHUB_API_KEY`
- `AZURE_MYSQL_HOST`
- `AZURE_MYSQL_USER`
- `AZURE_MYSQL_PASSWORD`

## 1. Setup and Config

In [4]:
#Check installed packages

!pip list

Package                                  Version
---------------------------------------- --------------------
absl-py                                  1.4.0
accelerate                               1.12.0
access                                   1.1.10.post3
affine                                   2.4.0
aiofiles                                 24.1.0
aiohappyeyeballs                         2.6.1
aiohttp                                  3.13.2
aiosignal                                1.4.0
aiosqlite                                0.22.0
alabaster                                1.0.0
albucore                                 0.0.24
albumentations                           2.0.8
ale-py                                   0.11.2
alembic                                  1.17.2
altair                                   5.5.0
annotated-doc                            0.0.4
annotated-types                          0.7.0
antlr4-python3-runtime                   4.9.3
anyio                        

In [6]:
# Install missing

# !pip install mysql-connector-python

In [7]:
# Import modules
import api
import db
import config
import pandas as pd

print(f"Symbols: {config.SYMBOLS}")
print(f"Date range: {config.FROM_DATE} to {config.TO_DATE}")

Symbols: ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 'AVGO', 'AMD', 'PLTR']
Date range: 2024-01-01 to 2026-01-04


## 2. Test Connections

In [8]:
# Test Finnhub API
print("Testing Finnhub API...")
api.test_connection()

Testing Finnhub API...
API connection successful


True

In [9]:
# Test MySQL Database
print("Testing MySQL connection...")
db.test_connection()

Testing MySQL connection...
Database connection successful


True

## 4. Historical Load (24 Months)
Run once for initial data population

In [10]:
# Fetch data from Finnhub API
print(f"Fetching data from {config.FROM_DATE} to {config.TO_DATE}...\n")

raw_data = api.fetch_all_symbols()

Fetching data from 2024-01-01 to 2026-01-04...

[1/10] Fetching AAPL... 245 transactions
[2/10] Fetching MSFT... 308 transactions
[3/10] Fetching GOOGL... 1180 transactions
[4/10] Fetching AMZN... 669 transactions
[5/10] Fetching NVDA... 1323 transactions
[6/10] Fetching META... 2000 transactions
[7/10] Fetching TSLA... 433 transactions
[8/10] Fetching AVGO... 317 transactions
[9/10] Fetching AMD... 488 transactions
[10/10] Fetching PLTR... 1191 transactions

Total: 8154 transactions from 10 symbols


In [11]:
# Transform to DataFrame
df = db.transform_to_dataframe(raw_data)

print(f"\nTransformed {len(df)} records")
print(f"\nColumns: {list(df.columns)}")

# Preview data
df.head(10)


Transformed 8154 records

Columns: ['symbol', 'insider_name', 'transaction_date', 'filing_date', 'transaction_code', 'shares', 'price', 'transaction_value', 'change_amount', 'filing_id']


,symbol,insider_name,transaction_date,filing_date,transaction_code,shares,price,transaction_value,change_amount,filing_id
0,AAPL,Adams Katherine L.,2025-11-12,2025-11-14,G,3750,NaN,NaN,-3750,0001462356-25-000012
1,AAPL,KONDO CHRIS,2025-11-07,2025-11-12,S,3752,271.23,1017654.96,-3752,0001631982-25-000011
2,AAPL,Parekh Kevan,2025-10-16,2025-10-17,S,500,248.73,124365.00,-500,0002050912-25-000008
3,AAPL,Parekh Kevan,2025-10-16,2025-10-17,S,1534,247.82,380155.88,-1534,0002050912-25-000008
4,AAPL,Parekh Kevan,2025-10-16,2025-10-17,S,1665,247.04,411321.60,-1665,0002050912-25-000008
5,AAPL,Parekh Kevan,2025-10-16,2025-10-17,S,500,245.89,122945.00,-500,0002050912-25-000008
6,AAPL,Parekh Kevan,2025-10-15,2025-10-17,M,5111,NaN,NaN,-5111,0002050912-25-000008
7,AAPL,Parekh Kevan,2025-10-15,2025-10-17,M,5816,NaN,NaN,-5816,0002050912-25-000008
8,AAPL,Parekh Kevan,2025-10-15,2025-10-17,M,5530,NaN,NaN,-5530,0002050912-25-000008
9,AAPL,Parekh Kevan,2025-10-15,2025-10-17,F,8062,249.34,2010179.08,-8062,0002050912-25-000008


In [12]:
# Insert into database
print("Inserting into MySQL...")

result = db.insert_transactions(df)

print(f"\n Historical load complete!")
print(f"   Total records: {result['total']}")
print(f"   Inserted: {result['inserted']}")
print(f"   Skipped (duplicates): {result['skipped']}")

Inserting into MySQL...
Inserted: 7050, Skipped (duplicates): 1104

 Historical load complete!
   Total records: 8154
   Inserted: 7050
   Skipped (duplicates): 1104


## 4. View Data

In [13]:
# Database statistics
stats = db.get_stats()

print(f"Total transactions: {stats['total_transactions']}")

if 'date_range' in stats:
    print(f"Date range: {stats['date_range']['from']} to {stats['date_range']['to']}")

print("\nBy Symbol:")
for symbol, count in stats['by_symbol'].items():
    print(f"  {symbol}: {count}")

Total transactions: 7050
Date range: 2024-01-02 to 2025-12-31

By Symbol:
  META: 1735
  NVDA: 1273
  GOOGL: 1012
  PLTR: 922
  AMZN: 526
  AMD: 406
  TSLA: 346
  AVGO: 313
  MSFT: 305
  AAPL: 212


In [14]:
# Run custom SQL query
conn = db.get_connection()

query = """
SELECT
    symbol,
    COUNT(*) as total_transactions,
    SUM(CASE WHEN transaction_code = 'P' THEN shares ELSE 0 END) as shares_bought,
    SUM(CASE WHEN transaction_code = 'S' THEN shares ELSE 0 END) as shares_sold,
    ROUND(SUM(COALESCE(transaction_value, 0)), 2) as total_value
FROM insider_transactions
GROUP BY symbol
ORDER BY total_transactions DESC
"""

summary = pd.read_sql(query, conn)
conn.close()

print("Summary by Symbol")
print("=" * 60)
summary

/tmp/ipython-input-3551195353.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  summary = pd.read_sql(query, conn)


Summary by Symbol


,symbol,total_transactions,shares_bought,shares_sold,total_value
0,META,1735,0.0,2015896.0,1.456794e+09
1,NVDA,1273,0.0,25155841.0,4.602500e+09
2,GOOGL,1012,0.0,1747759.0,7.111707e+08
3,PLTR,922,10000.0,80051386.0,3.671283e+09
4,AMZN,526,0.0,100990964.0,1.947950e+10
5,AMD,406,13445.0,1773484.0,4.123039e+08
6,TSLA,346,2572732.0,2393950.0,1.455807e+11
7,AVGO,313,4550.0,3649722.0,1.297706e+09
8,MSFT,305,3842.0,679170.0,5.232339e+08
9,AAPL,212,0.0,2026698.0,7.231677e+08


## 5. Incremental Load (Daily Updates)
### Run as needed to fetch new transactions

In [18]:
from datetime import date, timedelta

# Fetch last 7 days
days_back = 7
from_date = (date.today() - timedelta(days=days_back)).isoformat()
to_date = date.today().isoformat()

print(f"Fetching {from_date} to {to_date}...\n")

# Fetch
raw_data = api.fetch_all_symbols(from_date=from_date, to_date=to_date)

# Transform
df = db.transform_to_dataframe(raw_data)

if df.empty:
    print("\nNo new transactions found")
else:
    # Insert into database
    result = db.insert_transactions(df)
    print(f"\nInserted: {result['inserted']}, Skipped: {result['skipped']}")
    print(f"Done")

Fetching 2025-12-28 to 2026-01-04...

[1/10] Fetching AAPL... 0 transactions
[2/10] Fetching MSFT... 0 transactions
[3/10] Fetching GOOGL... 6 transactions
[4/10] Fetching AMZN... 0 transactions
[5/10] Fetching NVDA... 0 transactions
[6/10] Fetching META... 2 transactions
[7/10] Fetching TSLA... 1 transactions
[8/10] Fetching AVGO... 0 transactions
[9/10] Fetching AMD... 0 transactions
[10/10] Fetching PLTR... 0 transactions

Total: 9 transactions from 10 symbols
Inserted: 0, Skipped (duplicates): 9

Inserted: 0, Skipped: 9
Done
